In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
from skimage.transform import resize

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *
from conversion import images_to_volume

Setting environment for AWS compute node


No vtk


In [2]:
# downsample = 1
# downsample = 32
# downsample = 8
# downsample=8 is the maximum. Exceeding it results in memory error.

In [6]:
# for stack in all_nissl_stacks:
# for stack in ['MD652', 'MD653', 'MD657', 'MD658']:
for stack in ['MD662']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for downsample in [8, 32]:
    for downsample in [32]:

        images = {}
        for sec in metadata_cache['valid_sections'][stack]:
            print sec
            mask = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec, prep_id=2)

            if downsample == 32:
#                 img_rgb = DataManager.load_image(stack, section=sec, resol='thumbnail', version='cropped_tif')
                img_rgb = DataManager.load_image_v2(stack, section=sec, resol='thumbnail', prep_id=2)
                img = img_as_ubyte(rgb2gray(img_rgb))
                img[~mask] = 0
            else:
#                 img_rgb = DataManager.load_image(stack, section=sec, resol='lossless', version='cropped_gray')
                img_rgb = DataManager.load_image_v2(stack, section=sec, resol='lossless', prep_id=2)
                if downsample == 1:
                    img = img_as_ubyte(rgb2gray(img_rgb)) 
                else:
                    img = img_as_ubyte(rgb2gray(img_rgb[::downsample, ::downsample]))
                    mask = resize(mask, img.shape[:2]).astype(np.bool)
                img[~mask] = 0

            images[sec-1] = img

        voxel_z_size = SECTION_THICKNESS/(XY_PIXEL_DISTANCE_LOSSLESS * downsample)

        volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
    #     volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=first_sec-1+10)
        print volume.shape

        vol_fp = DataManager.get_intensity_volume_filepath(stack=stack, downscale=downsample)
        create_if_not_exists(os.path.dirname(vol_fp))
        bp.pack_ndarray_file(volume, vol_fp)
        upload_to_s3(vol_fp)

        bbox_fp = DataManager.get_intensity_volume_bbox_filepath(stack=stack, downscale=downsample)
        np.savetxt(bbox_fp, np.array(volume_bbox, np.int)[None, :], fmt='%d')
        upload_to_s3(bbox_fp)

100
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F35-2017.06.05-17.08.18_MD662_1_0103_prep2_thumbnail.tif"
1

rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F37-2017.06.05-18.09.08_MD662_3_0111_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F37-2017.06.05-18.09.08_MD662_3_0111_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F37-2017.06.05-18.09.08_MD662_3_0111_prep2_thumbnail.tif"
109
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F38-2017.06.05-18.38.44_MD662_1_0112_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F38-2017.06.05-18.38.44_MD662_1_0112_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F38-2017.06.05-18.38.44_MD662_1_0112_prep2_thumbnail_mask.png"
r

117
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F40-2017.06.05-19.36.43_MD662_3_0120_prep2_thumbnail.tif"
1

rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F43-2017.06.05-21.05.21_MD662_2_0128_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F43-2017.06.05-21.05.21_MD662_2_0128_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F43-2017.06.05-21.05.21_MD662_2_0128_prep2_thumbnail.tif"
126
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F43-2017.06.05-21.05.21_MD662_3_0129_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F43-2017.06.05-21.05.21_MD662_3_0129_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F43-2017.06.05-21.05.21_MD662_3_0129_prep2_thumbnail_mask.png"
r

134
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F46-2017.06.05-22.36.28_MD662_2_0137_prep2_thumbnail.tif"
1

...

338
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F134-2017.06.07-21.04.24_MD662_2_0401_prep2_thumbnail.

rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F137-2017.06.07-22.31.13_MD662_1_0409_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F137-2017.06.07-22.31.13_MD662_1_0409_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F137-2017.06.07-22.31.13_MD662_1_0409_prep2_thumbnail.tif"
347
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F137-2017.06.07-22.31.13_MD662_2_0410_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F137-2017.06.07-22.31.13_MD662_2_0410_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F137-2017.06.07-22.31.13_MD662_2_0410_prep2_thumbnail_mask.

355
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F140-2017.06.07-23.55.43_MD662_1_0418_prep2_thumbnail.

rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F142-2017.06.08-19.12.20_MD662_3_0426_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F142-2017.06.08-19.12.20_MD662_3_0426_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F142-2017.06.08-19.12.20_MD662_3_0426_prep2_thumbnail.tif"
364
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F143-2017.06.08-00.52.06_MD662_1_0427_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F143-2017.06.08-00.52.06_MD662_1_0427_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F143-2017.06.08-00.52.06_MD662_1_0427_prep2_thumbnail_mask.

372
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail_mask.png" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail_mask.png" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail_mask/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail_mask.png"
rm -rf "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail.tif" && mkdir -p "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail"
aws s3 cp "s3://mousebrainatlas-data/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail.tif" "/shared/CSHL_data_processed/MD662/MD662_prep2_thumbnail/MD662&661-F145-2017.06.08-01.42.12_MD662_3_0435_prep2_thumbnail.

In [5]:
display_image(volume[200,:,:])

/shared/MouseBrainAtlas/reconstruct/tmp.jpg